In [173]:
import numpy as np
from typing import Union, Tuple, Self, Iterable
import inspect

In [174]:
RNG = np.random.default_rng()
DTYPE = 'float64' 

In [175]:
'''TODO 
- add in auto grad visulisation
- add in a way to test the primative operations (make sure log, exp and rtruediv pass)
- add in convolutions (possibly)
- todo implement dataloaders 
- add logging, WandB and also terminal logging
'''

class Tensor():
    def __init__(self, data, children=(), op=''):
        self.data: np.ndarray = np.array(data, dtype=DTYPE)
        self.grad = np.zeros_like(data, dtype=DTYPE)
        self._prev = set(children)
        self._backward = lambda : None
        self._op = op

    @property
    def shape(self) -> Tuple[int]:
        return self.data.shape
    
    @property
    def size(self) -> int: 
        return self.data.size
    
    def zero_grad(self) -> None:
        self.grad = np.zeros_like(self.data, dtype=DTYPE)

    def item(self) -> np.ndarray:
        return self.data
    
    def _unbroadcast(self, grad: np.ndarray) -> Self:
        dims_to_remove = tuple(i for i in range(len(grad.shape) - len(self.shape))) 
        # remove prepended padding dimensions
        grad = np.sum(grad, axis=dims_to_remove, keepdims=False) 
        dims_to_reduce = tuple(i for i, (d1,d2) in enumerate(zip(grad.shape, self.shape)) if d1!=d2)
        # reduce broadcasted dimensions
        return np.sum(grad, axis=dims_to_reduce, keepdims=True)

    # need to build topo graph and then go through it and call backwards on each of the tensors
    def backward(self) -> None:
        self.grad = np.ones_like(self.data)
        topo = []
        visited = set()

        # do DFS on un-visited nodes, add node to topo-when all its children have been visited
        def build_topo(node):
            if node not in visited:
                visited.add(node)
                for child in node._prev:
                    build_topo(child)
                topo.append(node)
        build_topo(self)

        for node in reversed(topo):
            node._backward()
            
    def __add__(self, rhs) -> Self:
        rhs = rhs if isinstance(rhs, Tensor) else Tensor(rhs)
        out = Tensor(self.data + rhs.data, (self, rhs), '+')

        def _backward():
            self.grad += self._unbroadcast(out.grad)
            rhs.grad += rhs._unbroadcast(out.grad)
        out._backward = _backward
        return out
    
    def __neg__(self) -> Self:
        out = Tensor(-self.data, (self,), 'neg')

        def _backward():
            self.grad += -out.grad
        out._backward = _backward
        return out
    
    def __sub__(self, rhs) -> Self:
        return self + (-rhs)

    def __mul__(self, rhs) -> Self:
        rhs = rhs if isinstance(rhs, Tensor) else Tensor(rhs)
        out = Tensor(self.data*rhs.data, (self, rhs), f'*')

        def _backward():
            self.grad += self._unbroadcast(out.grad * rhs.data)
            rhs.grad += rhs._unbroadcast(out.grad * self.data)
        out._backward = _backward
        return out
        
    def __truediv__(self, rhs) -> Self:
        return self * (rhs**-1)
      
    def __pow__(self, rhs) -> Self: 
        rhs = rhs if isinstance(rhs, Tensor) else Tensor(rhs)
        lhs_is_neg = self.data < 0
        rhs_is_frac = ~np.isclose(rhs.data % 1, 0)
        if np.any(lhs_is_neg & rhs_is_frac):
            raise ValueError('cannot raise negative value to a decimal power')
        
        out = Tensor(self.data**rhs.data, (self, rhs), f'**')

        def _backward():
            self.grad += out.grad * ((rhs.data)*(self.data**(rhs.data-1)))
            rhs.grad += out.grad * (self.data ** rhs.data) * np.log(rhs.data)
        out._backward = _backward
        return out
    
    '''data shape: (da, ..., d2, d1, n, k) rhs shape: (ob, ..., o2, o1, k, m)
       inputs are broadcast so that they have the same shape by expanding along
       dimensions if possible, out shape: (tc, ..., t2, t1, n, m), where ti = max(di, oi)
       if di or oi does not exist it is treated as 1, and c = max d, a
       if self is 1d shape is prepended with a 1, for rhs it would be appended'''
    def __matmul__(self, rhs) -> Self:
        rhs = rhs if isinstance(rhs, Tensor) else Tensor(rhs)
        out = Tensor(self.data @ rhs.data, (self, rhs), '@')

        def _backward():
            A, B, = self.data, rhs.data
            g = out.grad
            # broadcast 1d arrays to be 2d 
            A2 = A.reshape(1, -1) if len(A.shape) == 1 else A
            B2 = B.reshape(-1, 1) if len(B.shape) == 1 else B
            # extend g to have reduced dims
            g = np.expand_dims(g, -1) if len(B.shape) == 1 else g
            g = np.expand_dims(g, -2) if len(A.shape) == 1 else g
            # transpose last 2 dimensions, as matmul treats tensors as batched matricies
            self.grad += self._unbroadcast(g @ B2.swapaxes(-2, -1))
            rhs.grad += rhs._unbroadcast(A2.swapaxes(-2, -1) @ g)
        out._backward = _backward
        return out

    def relu(self) -> Self:
        out = Tensor((self.data > 0) * self.data, (self,), 'Relu')

        def _backward():
            self.grad += (self.data > 0) * out.grad
        out._backward = _backward
        return out
    
    # need to check inp is non-negative
    def log(self) -> Self:
        if np.any(self.data < 0):
            raise ValueError('cannot log negative values')
        out = Tensor(np.log(self.data), (self,), 'log')

        def _backward():
            self.grad += 1 / self.data 
        out._backward = _backward
        return out
    
    def exp(self) -> Self:
        out = Tensor(np.exp(self.data), (self,), 'exp')

        def _backward():
            self.grad += np.exp(self.data)
        out._backward = _backward
        return out
    
    def sum(self, axis=None) -> Self:
        out = Tensor(np.sum(self.data, axis=axis), (self,), 'sum')

        def _backward():
            g = np.expand_dims(out.grad, axis) if axis is not None else out.grad
            self.grad += g
        out._backward = _backward
        return out

    def mean(self, axis=None) -> Self:
        out = Tensor(np.mean(self.data, axis=axis), (self,), 'mean')

        def _backward():
            N =  self.size // out.size 
            g = np.expand_dims(out.grad, axis) if axis is not None else out.grad
            self.grad += g / N
        out._backward = _backward
        return out
    
    def __radd__(self, lhs) -> Self:
        return self + lhs
    
    def __rsub__(self, lhs) -> Self:
        return self + lhs
    
    def __rmul__(self, lhs) -> Self:
        return self * lhs
    
    def __rtruediv__(self, lhs) -> Self:
        try:
            lhs = Tensor(lhs)
        except TypeError:
            return NotImplementedError
        return lhs / self
    
    def __rpow__(self, lhs) -> Self:
        try:
            lhs = Tensor(lhs)
        except TypeError:
            return NotImplementedError
        return lhs ** self
    
    def __rmatmul__(self, lhs) -> Self:
        try:
            lhs = Tensor(lhs)
        except TypeError:
            return NotImplementedError
        return lhs @ self
    
    @classmethod
    def random(cls, shape: tuple, bounds = (0,1)) -> Self:
        lower, upper = bounds
        data = RNG.random(shape, dtype=DTYPE)*(upper-lower) + lower
        return cls(data)
    
    def __repr__(self) -> str:
        return f'tensor shape: {self.shape}, op:{self._op}'        


In [176]:
class Parameter(Tensor):
    def __init__(self, data):
        super().__init__(data)
    
    @classmethod
    def kaiming(cls, fan_in, shape):
        std = np.sqrt(2/fan_in)
        weights = RNG.standard_normal(shape, dtype=DTYPE)*std
        return cls(weights)
    
    @classmethod
    def zeros(cls, shape):
        return cls(np.zeros(shape, dtype=DTYPE))
    
    def __repr__(self) -> str:
        return f'parameter shape: {self.shape}, size: {self.size}' 

In [177]:
from abc import ABC, abstractmethod

class Module(ABC):
    
    def __call__(self, input: Tensor) -> Tensor:
        return self.forward(input)
    
    @property
    def modules(self) -> list[Self]:
        modules: list[Self] = []
        for value in self.__dict__.values():
            if isinstance(value, Module):
                modules.append(value)

            elif isinstance(value, dict):
                for v in value.values():
                    if isinstance(v, Module):
                        modules.append(v)

            elif isinstance(value, Iterable) and not isinstance(value, (str, bytes)):
                for v in value:
                    if isinstance(v, Module):
                        modules.append(v)
                    
        return modules
    
    @property
    def params(self) -> list[Parameter]:
        immediate_params = [attr for attr in self.__dict__.values() 
                                    if isinstance(attr, Parameter)]
        modules_params = [param for module in self.modules 
                                    for param in module.params]
        return immediate_params + modules_params
    
    @abstractmethod
    def forward(self, input: Tensor) -> Tensor:
        pass
    
    def zero_grad(self) -> None:
        for param in self.params:
            param.zero_grad()

class Sequential(Module):
    def __init__(self, layers):
        self.layers = layers
    
    def forward(self, input: Tensor) -> Tensor:
        x = input
        for layer in self.layers:
            x = layer(x)
        return x
    
class Affine(Module):
    def __init__(self, in_dim, out_dim):
        self.A = Parameter.kaiming(in_dim, (in_dim, out_dim))
        self.b = Parameter.zeros((out_dim))

    def forward(self, x):
        # x: (B, in), A : (in, out), B: out
        return (x @ self.A) + self.b

class Relu(Module):
    def forward(self, x):
        return x.relu()

In [178]:
class SoftMaxCrossEntropy():

    def __call__(z: Tensor, y) -> Tensor:
        '''logits z, shape (B, C), true lables y, shape (B, C)'''
        loss = ((z * y).sum(axis=-1) + ((z.exp()).sum(axis=-1)).log()).mean()
        return loss

class SGD():
    def __init__(self, params: list[Parameter], lr: float):
        self.lr = lr
        self.params = params
    
    def step(self) -> None:
        for param in self.params:
            param.data += -self.lr * param.grad

In [179]:
class Trainer():
    def __init__(self, model, optimiser, loss, train_loader, test_loader, logger, wandb_run = None):
        self.model = model
        self.optimiser = optimiser
        self.loss = loss
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.epoch = 1
        self.logger = logger
        self.wandb_run = wandb_run

    def train_epoch():
        pass

    def validate():
        pass
    
    def fit():
        pass
    
    def log_metrics():
        pass

In [180]:
feedforward = Sequential([Affine(50, 100), Relu(), Affine(100, 200), Relu(), Affine(200, 10)])
feedforward.params

[parameter shape: (50, 100), size: 5000,
 parameter shape: (100,), size: 100,
 parameter shape: (100, 200), size: 20000,
 parameter shape: (200,), size: 200,
 parameter shape: (200, 10), size: 2000,
 parameter shape: (10,), size: 10]

In [181]:
''' auto-grad testing suite
    TODO:
    - test all of the auto-grad primatives, 
    - test using central differences
    - test by modifying each parameter individually i.e. only do scalar pertubations
'''
RNG = np.random.default_rng()
if DTYPE=='float64':
    EPS, ATOL, RTOL = 1e-6, 1e-5, 1e-3
else:
    EPS, ATOL, RTOL = 1e-4, 1e-4, 1e-2
K = 20

def compute_central_diff_error(test_fn, test_input, 
            other_inputs, eps, perturbed_idx, tols):
    '''verify auto-grad of funciton f: R^n -> R'''
    atol, rtol = tols

    # rescale epsilon and convert to tensor
    perturbed_val = test_input.data[perturbed_idx]
    eps = eps * (1 + abs(perturbed_val))
    pertubation_tensor = np.zeros_like(test_input.data, dtype=DTYPE)
    pertubation_tensor[perturbed_idx] += eps 
    pertubation_tensor = Tensor(pertubation_tensor)

    # Compute grad
    for tensor in [test_input, *other_inputs]:
        tensor.zero_grad()
    clean_out = test_fn(test_input, other_inputs)
    clean_out.backward()
    auto_grad = test_input.grad[perturbed_idx]

    # Compute central diff Grad approximaiton
    test_forward = test_input + pertubation_tensor
    forward_out = test_fn(test_forward, other_inputs).item()
    test_back = test_input - pertubation_tensor
    back_out = test_fn(test_back, other_inputs).item()
    approx_grad = (forward_out - back_out) / (2*eps)


    abs_err = abs(approx_grad - auto_grad)
    rel_err = abs_err / (abs(auto_grad) + atol)
    is_close = abs_err <= atol + rtol*abs(auto_grad)

    return is_close, abs_err, rel_err, clean_out.item(), forward_out, back_out

# need to generate inputs, compute cd err and output/format test result, to log file maybe?
def test_fn_random_inputs(test_fn, test_shape, other_shapes=[], input_bounds=(-10, 10),
                          num_samples=K, eps=EPS, tols=(ATOL, RTOL)):
    
    test_input = Tensor.random(test_shape, input_bounds)
    other_inputs = [Tensor.random(shape, input_bounds) for shape in other_shapes]

    num_samples = min(test_input.size, num_samples)
    pertubation_nums = RNG.choice(test_input.size, size=num_samples, replace=False)
    pretubation_idxs = np.unravel_index(pertubation_nums, test_shape)

    all_close = True
    failed = 0
    # log = inspect.getsource(test_fn) + '\n' 
    log = ''
    log += f'test input \n {test_input.data} \nother inputs \n'
    for other_input in other_inputs:
        log += f' {other_input.data} \n'
    for sample_i in range(num_samples):
        perturbed_idx = tuple(int(pert_dim[sample_i]) for pert_dim in pretubation_idxs)
        is_close, abs_err, rel_err, clean_out, forward_out, back_out = compute_central_diff_error(
                                        test_fn, test_input, other_inputs, eps, perturbed_idx, tols)
        log += f'test {'passed' if is_close else 'failed'}: abs err = {abs_err:.4f}, rel err = {rel_err:.4f}, perturbed idx = {perturbed_idx} \n'
        log += f'clean_out: {clean_out} forward_out: {forward_out} back_out: {back_out} \n'
        if not is_close:
            all_close = False
            failed += 1
            # some logic for logging the failed case

    return all_close, log
        

In [182]:
bin_ufuncs = {'add' : lambda test_inp, other_inps: (test_inp+other_inps[0]).sum(),
            'radd': lambda test_inp, other_inps: (other_inps[0]+test_inp).sum(),
            'sub' : lambda test_inp, other_inps: (test_inp-other_inps[0]).sum(),
            'rsub': lambda test_inp, other_inps: (other_inps[0]-test_inp).sum(),
            'mul' : lambda test_inp, other_inps: (test_inp*other_inps[0]).sum(),
            'rmul': lambda test_inp, other_inps: (other_inps[0]*test_inp).sum(),
            # 'pow' : lambda test_inp, other_inps: (test_inp**other_inps[0]).sum(),
            # 'rpow': lambda test_inp, other_inps: (other_inps[0]**test_inp).sum(),
            'truediv' : lambda test_inp, other_inps: (test_inp/other_inps[0]).sum(),
            'rtruediv': lambda test_inp, other_inps: (other_inps[0]/test_inp).sum(),
            }

matmul_fns = {'matmul': lambda test_inp, other_inps: (test_inp@other_inps[0]).sum(),
              'rmatmul': lambda test_inp, other_inps: (other_inps[0]@test_inp).sum(),}

unary_ufunc = {'relu': lambda test_inp, other_inps: (test_inp.relu()).sum(),
            'log': lambda test_inp, other_inps: (test_inp.log()).sum(),
            'exp': lambda test_inp, other_inps: (test_inp.exp()).sum(),
            'sum': lambda test_inp, other_inps: test_inp.sum(),
            'mean': lambda test_inp, other_inps: test_inp.mean(),}
                      

In [183]:
for func_name, test_fn in unary_ufunc.items():
    test_shape, other_shapes = (2, 3), [(3,2)]
    input_bounds = (1, 100) if func_name == 'log' else (-100, 100)
    all_close, log = test_fn_random_inputs(test_fn, test_shape, other_shapes, input_bounds=input_bounds)
    print(f'function: {func_name} {'passed' if all_close else 'failed'}')
    if not all_close:
        print(log)

for func_name, test_fn in matmul_fns.items():
    test_shape = (2, 3) if func_name == 'matmul' else (3, 2)
    other_shapes = [(3, 2)] if func_name == 'matmul' else [(2, 3)]
    all_close, log = test_fn_random_inputs(test_fn, test_shape, other_shapes, input_bounds=input_bounds)
    print(f'function: {func_name} {'passed' if all_close else 'failed'}')
    if not all_close:
        print(log)

for func_name, test_fn in bin_ufuncs.items():
    test_shape, other_shapes = (2, 3), [(2,3)]
    all_close, log = test_fn_random_inputs(test_fn, test_shape, other_shapes, input_bounds=input_bounds)
    print(f'function: {func_name} {'passed' if all_close else 'failed'}')
    if not all_close:
        print(log)

function: relu passed
function: log passed
function: exp passed
function: sum passed
function: mean passed
function: matmul passed
function: rmatmul passed
function: add passed
function: radd passed
function: sub passed
function: rsub passed
function: mul passed
function: rmul passed


C:\Users\nikiw\AppData\Local\Temp\ipykernel_40488\3619599098.py:102: RuntimeWarning: invalid value encountered in log
  rhs.grad += out.grad * (self.data ** rhs.data) * np.log(rhs.data)


ValueError: non-broadcastable output operand with shape () doesn't match the broadcast shape (2,3)

In [ ]:
feedforward = Sequential([Affine(50, 100), Relu(), Affine(100, 200), Relu(), Affine(200, 10)])
test_shape, other_shapes = (50, 50), (1,)
test_fn = lambda test, other: feedforward(test).sum()
all_close, log = test_fn_random_inputs(test_fn, test_shape, other_shapes)

print(all_close)
print(log)

True
test input 
 [[-9.71146134 -8.89748529 -0.72254851 ...  6.45520874  8.76632988
   0.14466156]
 [-5.72667199 -5.32969713 -0.88090023 ... -2.80718778  2.35732905
  -9.82730865]
 [ 0.03189104  3.2142204  -6.39239274 ...  3.12862717 -1.14919706
  -7.33857508]
 ...
 [ 4.49626838 -3.78306744  4.85917813 ...  3.27465866  7.58974102
  -4.54765277]
 [ 4.63381445  5.7512736  -8.47256845 ... -1.67872051 -6.61579239
  -3.042671  ]
 [ 6.90589474 -7.29616348  8.74373567 ...  0.9312988  -1.52038462
  -1.96318709]] 
other inputs 
 [-9.30978657] 
test passed: abs err = 0.0000, rel err = 0.0000, perturbed idx = (23, 35) 
clean_out: -251.5008700293402 forward_out: -251.5008700339569 back_out: -251.50087002472355 
test passed: abs err = 0.0000, rel err = 0.0000, perturbed idx = (7, 40) 
clean_out: -251.5008700293402 forward_out: -251.50086482080687 back_out: -251.50087523787354 
test passed: abs err = 0.0000, rel err = 0.0000, perturbed idx = (47, 37) 
clean_out: -251.5008700293402 forward_out: -251.

In [213]:
A = Tensor.random((2,3))
C = Tensor.random((2,3))
B = (A ** C).sum()
B.backward()
print(B.data)
print(A.grad)
print(C.grad)

4.144422779277987
[[0.65209104 1.09494424 0.08690697]
 [1.33157557 0.65336944 0.44156278]]
[[-1.26670726 -0.04725824 -3.08710802]
 [-0.46560782 -0.68132569 -0.8277387 ]]
